In [388]:
import os
import pandas as pd

In [389]:
path = os.getcwd()

In [390]:
path

'C:\\Users\\USER\\Desktop\\linchengwei_linkpred'

In [391]:
journal = input("scientometrics / jasist: ")
entity = input("co-author / co-occurrence: ")

scientometrics / jasist: scientometrics
co-author / co-occurrence: co-occurrence


In [392]:
data_path = path+"\\data\\{}\\".format(journal)
data_path

'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data\\scientometrics\\'

In [393]:
times = os.listdir(data_path)
times

['2010-2015', '2016-2016', '2016-2017', '2016-2018', '2016-2019']

In [394]:
data_time_paths = [data_path+t for t in times]
data_time_paths

['C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data\\scientometrics\\2010-2015',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data\\scientometrics\\2016-2016',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data\\scientometrics\\2016-2017',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data\\scientometrics\\2016-2018',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data\\scientometrics\\2016-2019']

# read data from data_transformation_output

In [395]:
data_path = path+"\\data_transformation_output\\{}\\wos_csv\\".format(journal)
data_path_origin = [data_path+f for f in os.listdir(data_path) if f.endswith('.csv')]

data = dict()
for i, time in enumerate(times):
    print('-'*10, time)
    data[time] = pd.read_csv(data_path_origin[i], index_col=0)
    
data_path_origin

---------- 2010-2015
---------- 2016-2016
---------- 2016-2017
---------- 2016-2018
---------- 2016-2019


['C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\2010-2015.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\2016-2016.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\2016-2017.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\2016-2018.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\2016-2019.csv']

In [396]:
data_path_stem = [data_path+"stem\\"+f for f in os.listdir(data_path+"stem")]

data_stem = dict()
for i, time in enumerate(times):
    print('-'*10, time)
    data_stem[time] = pd.read_csv(data_path_stem[i], index_col=0)
    
data_path_stem

---------- 2010-2015
---------- 2016-2016
---------- 2016-2017
---------- 2016-2018
---------- 2016-2019


['C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\stem\\2010-2015.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\stem\\2016-2016.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\stem\\2016-2017.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\stem\\2016-2018.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\stem\\2016-2019.csv']

In [397]:
data_path_lemma = [data_path+"lemma\\"+f for f in os.listdir(data_path+"lemma")]

data_lemma = dict()
for i, time in enumerate(times):
    print('-'*10, time)
    data_lemma[time] = pd.read_csv(data_path_lemma[i], index_col=0)
    
data_path_lemma

---------- 2010-2015
---------- 2016-2016
---------- 2016-2017
---------- 2016-2018
---------- 2016-2019


['C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\lemma\\2010-2015.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\lemma\\2016-2016.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\lemma\\2016-2017.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\lemma\\2016-2018.csv',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\wos_csv\\lemma\\2016-2019.csv']

# transform to node and edge

In [347]:
# generate all nodes
def generate_nodes(df, col):
    afs = list(df[col])
    afs_list = []
    for af in afs:
        if type(af)==str:
            afs_list.append(af.split('; '))
        else:
            afs_list.append('')
            # print(af)
    all_afs = sorted(list(set(([name for names in afs_list for name in names]))))
    return all_afs, afs_list

def generate_all_nodes(data, col):
    all_nodes = []
    nodes = dict()
    nodes_list = dict()
    for i, time in enumerate(times):
        df = data[time]
        nodes[time], nodes_list[time] = generate_nodes(df, col)
        all_nodes += nodes[time]
        all_nodes = sorted(list(set(all_nodes)))

        print(time, len(nodes[time]), len(all_nodes)) 
    
    return nodes, nodes_list, all_nodes

In [277]:
def generate_df_nodes(nodes, all_nodes):
    df_nodes = dict()
    for i, time in enumerate(times):
        
        df_node = pd.DataFrame(columns=['id', 'label'])
        node_id_list, node_label_list = [], []

        for name in nodes[time]:
            node_id_list.append(all_nodes.index(name)+1)
            node_label_list.append(name)

        df_node['id'] = node_id_list
        df_node['label'] = node_label_list

        df_nodes[time] = df_node  

    return df_nodes

In [285]:
def generate_node_id(nodes_list, all_nodes):
    nodes_id = dict()
    
    for i, time in enumerate(times):
        
        node_id_list = []
        for li in nodes_list[time]:

            id_li = []
            for word in li:
                word_id = all_nodes.index(word)+1
                id_li.append(word_id)

            node_id_list.append(id_li)
            
        nodes_id[time] = node_id_list
    return nodes_id

In [294]:
def generate_nodes_id_pair_list(nodes_id):
    nodes_id_pair_list = dict()
    for i, time in enumerate(times):
        all_pair_list = []
        for cooccurr in nodes_id[time]:
            pair_list = []
            for pair in itertools.combinations(cooccurr, 2):
                pair_list.append(pair)

            all_pair_list.append(pair_list)
        nodes_id_pair_list[time] = all_pair_list
    return nodes_id_pair_list

In [312]:
def generate_df_edges(nodes_id_pair_list):    
    df_edges = dict()
    for i, time in enumerate(times):
        all_pairs = []
        for pairs in nodes_id_pair_list[time]:
            all_pairs+=pairs

        source_list, target_list = [], []
        df_edge = pd.DataFrame(columns=['source', 'target', 'weight'])

        for pair in all_pairs:
            source_list.append(pair[0])
            target_list.append(pair[1])

        df_edge['source'] = source_list
        df_edge['target'] = target_list
        df_edge['weight'] = [1]*len(all_pairs)
        df_edge = df_edge.sort_values(by=['source', 'target'])
        df_edge.index = list(range(len(df_edge)))

        df_edges[time] = df_edge

    return df_edges

In [321]:
def generate_weight(df_edges):
    # x = pd.DataFrame(columns=['source', 'target', 'weight'])
    # x['source'] = [1,1,1,2,3,3,4,5,5,5]
    # x['target'] = [2,2,2,3,4,4,5,1,1,1]
    # x['weight'] = [1,1,1,1,1,1,1,1,1,1]
    # x = df_edges[0][['source', 'target', 'weight']]
    
    df_weighted_edges = dict()
    for i, time in enumerate(times):
        df = df_edges[time].copy()
        x = df[['source', 'target', 'weight']]
        x.loc[len(x.index)] = ['end', 'end', 1] # use end index to subtract the weight by index number in the end

        y = x.drop_duplicates(subset=['source', 'target'])
        y_index = list(y.index)
        # print(y_index)
        new_weight = []
        for i, index in enumerate(y_index):
            if i+1<len(y_index):
                new_weight.append(y_index[i+1]-y_index[i])
        y = y[y['source']!='end']
        y['weight'] = new_weight
        y = y.sort_values(by=['source', 'target'])
        df_weighted_edges[time] = y
        
    return df_weighted_edges

In [350]:
def main(nodes, nodes_list, all_nodes):
    df_nodes = generate_df_nodes(nodes, all_nodes)
    nodes_id = generate_node_id(nodes_list, all_nodes)
    nodes_id_pair_list = generate_nodes_id_pair_list(nodes_id)
    df_edges = generate_df_edges(nodes_id_pair_list)
    df_weighted_edges = generate_weight(df_edges)
    
    return df_nodes, df_weighted_edges

## main

In [398]:
if entity=='co-occurrence':
    col = 'DE'
    print('co-occurrence')
    print('-'*20, 'origin')
    nodes, nodes_list, all_nodes = generate_all_nodes(data, col)

    print('\n', '-'*20, 'lemma')
    nodes_lemma, nodes_list_lemma, all_nodes_lemma = generate_all_nodes(data_lemma, col)

    print('\n', '-'*20, 'stem')
    nodes_stem, nodes_list_stem, all_nodes_stem = generate_all_nodes(data_stem, col)
elif entity=='co-author':
    col = 'AF'
    print("co-author")
    print('-'*20)
    nodes, nodes_list, all_nodes = generate_all_nodes(data, col)

co-occurrence
-------------------- origin
2010-2015 4351 4351
2016-2016 1223 5113
2016-2017 2349 5975
2016-2018 3364 6816
2016-2019 4172 7512

 -------------------- lemma
2010-2015 4097 4097
2016-2016 1190 4820
2016-2017 2256 5618
2016-2018 3208 6400
2016-2019 3963 7042

 -------------------- stem
2010-2015 4010 4010
2016-2016 1178 4709
2016-2017 2221 5483
2016-2018 3148 6245
2016-2019 3884 6867


In [399]:
def generate_output_dir(output_path):
    for time in times:
        try:
            os.mkdir(output_path+time)
        except FileExistsError:
            print("File Existed!!")
            
    output_path = [output_path+t for t in os.listdir(output_path) if t.startswith('20')]
    return output_path

output_path = path+"\\data_transformation_output\\{}\\{}\\".format(journal, entity)
output_lemma_path = path+"\\data_transformation_output\\{}\\{}\\lemma\\".format(journal, entity)
output_stem_path = path+"\\data_transformation_output\\{}\\{}\\stem\\".format(journal, entity)

output_lemma_path = generate_output_dir(output_lemma_path)
output_lemma_path

['C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\co-occurrence\\lemma\\2010-2015',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\co-occurrence\\lemma\\2016-2016',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\co-occurrence\\lemma\\2016-2017',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\co-occurrence\\lemma\\2016-2018',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\co-occurrence\\lemma\\2016-2019']

In [401]:
output_stem_path = generate_output_dir(output_stem_path)
output_stem_path

['C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\co-occurrence\\stem\\2010-2015',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\co-occurrence\\stem\\2016-2016',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\co-occurrence\\stem\\2016-2017',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\co-occurrence\\stem\\2016-2018',
 'C:\\Users\\USER\\Desktop\\linchengwei_linkpred\\data_transformation_output\\scientometrics\\co-occurrence\\stem\\2016-2019']

In [402]:
def save_files(nodes, nodes_list, all_nodes, output_path):
    df_nodes, df_weighted_edges = main(nodes, nodes_list, all_nodes)

    for i, time in enumerate(times):
        df_nodes[time].to_csv(output_path[i]+'\\node.csv')
        df_weighted_edges[time].to_csv(output_path[i]+'\\edge.csv')
        
save_files(nodes_stem, nodes_list_stem, all_nodes_stem, output_stem_path)